In [ ]:
!pip3 install python-sat

In [ ]:
import numpy as np
import time
from random import randrange
from random import seed
from random import shuffle
from pysat.solvers import Glucose4
from pysat.solvers import Minisat22
from pysat.solvers import MapleCM
from pysat.solvers import Lingeling
from pysat.solvers import Minicard
import matplotlib.pyplot as plt
import math
import networkx as nx
import random
import sympy
from sympy.logic import POSform
from sympy import symbols
w, x, y, z, a = symbols('w x y z a')

In [ ]:
def print_grid(grid, m):
    count = 0
    for elem in grid:
        print(elem, end=" ")
        count = (count + 1)%m
        if count==0:
            print()
    print()

class Graph:

    def __init__(self, n):
        self.n = n
        self.adj = [[] for i in range(n)]
        self.degree = 0    #maximum degree
        self.negative = set()

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)
        if len(self.adj[u])>self.degree or len(self.adj[v])>self.degree:
            self.degree+=1

    #all zero state
    def find_coloring(self):
        colors = [0 for _ in range(self.n)]
        return colors


    '''
        Forward simulation of the heat-bath dynamics
    '''

    def heatbath_dynamics(self, coloring, transitions):

        for (node, interval) in reversed(transitions):

            count = 0
            for u in self.adj[node]:
                negate = 1 if (node, u) in graph.negative else 0
                count += (coloring[u] + negate) % 2

            if count >= interval:
                coloring[node] = 1
            else:
                coloring[node] = 0

        return coloring



    # Return a set 'fam' of vertices with distance at most R from v
    # and a set 'brdr' of all vertices in 'fam' with neighbors outside of 'fam'.

    def family(self, v, R):

        visited = {v}   # Set of visited nodes.
        queue = [v]     # Initialize a queue

        for _ in range(R):
            new_queue = []
            for u in queue:
                for neigh in self.adj[u]:
                    if neigh not in visited:
                        visited.add(neigh)
                        new_queue.append(neigh)
            queue = new_queue.copy()

        border = set()
        for u in queue:
            for neigh in self.adj[u]:
                if neigh not in visited:
                    border.add(u)
                    break

        return visited, border

In [ ]:
#clauses to ensure that: the number of neighbours of v with spin 1
# is larger than 'interval' <=> the spin of v is set to 1
def interval_to_clauses(interval):

    #define minterms of clause
    minterms = []
    for X in [0,1]:
      for Y in [0,1]:
        for Z in [0,1]:
          for W in [0,1]:
            if X+Y+Z+W>=interval:
                A = 1
            else:
                A = 0
            minterms.append([X,Y,Z,W,A])

    #print(minterms)
    expression = str(POSform([w, x, y, z, a], minterms))
    #print(expression)

    # Define the variable map
    variable_map = {'w': 1, 'x': 2, 'y': 3, 'z': 4, 'a': 5}
    # Remove spaces for easier processing
    expression = expression.replace(" ", "")
    # Split by '&' to get individual clauses
    clauses = expression.split('&')
    result = []

    for clause in clauses:
        # Remove outer parentheses
        clause = clause.strip('()')
        # Split by '|'
        literals = clause.split('|')
        clause_list = []

        for literal in literals:
            # Handle negation
            if literal.startswith('~'):
                clause_list.append(-variable_map[literal[1]])  # Negated variable
            else:
                clause_list.append(variable_map[literal])  # Positive variable
        result.append(clause_list)

    return result

learnt_intervals = [interval_to_clauses(i) for i in range(0,6)]

#print(interval_to_clauses(2))

#checks whether a collapse has occured
def CFTPcollapse_ising(graph, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i):
        return i + 1
    #color after transition k
    def P(k):
        return n + k + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring()
    #print("primary coloring:", init_state)
    final_state = graph.heatbath_dynamics(init_state.copy(), transitions)
    #print_grid(final_state,m)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()


    for k in range(t):

        node, interval = transitions[-k-1] # apply the transitions in reverse order

        clauses = learnt_intervals[interval]

        variable_map = {}
        # If S(u)=1 for some neighbour u, P(k) is set to 0
        curr = 1

        for u in graph.adj[node]:
            negate = -1 if (node, u) in graph.negative else 1
            if (d[u] == -1):
                variable_map[curr]=S(u)*negate
                variable_map[-curr]=-S(u)*negate
            else:
                variable_map[curr]=P(d[u])*negate
                variable_map[-curr]=-P(d[u])*negate
            curr+=1

        variable_map[5]=P(k)
        variable_map[-5]=-P(k)

        #print(variable_map)
        #print(clauses)
        new_clauses = [[variable_map[literal] for literal in clause ] for clause in clauses ]
        #print(new_clauses)

        for clause in new_clauses:
            g.add_clause(clause)

        d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(n):
        if (d[i] == -1):
            clause.append(-(final_state[i]*2-1)*S(i))
        else:
            clause.append(-(final_state[i]*2-1)*P(d[i]))

    g.add_clause(clause)

    time_solver = time.time()
    # Solve the CSP
    status = g.solve()
    print("SAT solver time:", time.time() - time_solver)

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]
        #print(model)
        for u in range(n):
            if model[u]>0:
                coloring[u] = 1
            else:
                coloring[u] = 0
        #print(coloring)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


def find_interval(u, lamda):
    a, b, c, d, e = 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1)

    #(start, end, label)
    intervals = [
        (0, a, 0),
        (a, b, 1),
        (b, c, 2),
        (c, d, 3),
        (d, e, 4),
        (e, 1, 5),
    ]

    for start, end, label in intervals:
        if start <= u < end:
            return label

    print("ERROR")
    return


#lamda>1
def CFTPsolver_ising(graph, lamda, solver_name):
    n = graph.n
    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = randrange(n)
            u = random.uniform(0, 1)
            interval = find_interval(u,lamda)
            transitions.append((node, interval))

        time_before = time.time()

        print(t)

        ret_value = CFTPcollapse_ising(graph, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.heatbath_dynamics(coloring1, transitions)
            coloring2 = graph.heatbath_dynamics(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")
            if coloring1 == coloring2:
                print_grid(coloring1,m)

        print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
#Construct a periodic m x m grid

m = 20
n = m*m

graph = Graph(n)
seed(10)
prob = 0.11
def add_negative(x,y):
    u = random.uniform(0, 1)
    if u < prob:
        graph.negative.add((x, y))
        graph.negative.add((y, x))

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)
        if i<m:
            add_negative(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)
        if j<m:
            add_negative(m*i + j, m*i+m + j)

for j in range(m):
    graph.add_edge(m*j, m*j+m-1)
    if i<m:
        add_negative(m*j, m*j+m-1)

for i in range(m):
    graph.add_edge(i, m*(m-1) + i)
    if j<m:
        add_negative(i, m*(m-1) + i)

print(graph.negative)
print(len(graph.negative)//2)

{(96, 95), (256, 236), (384, 4), (18, 17), (174, 194), (67, 68), (78, 77), (348, 328), (55, 75), (79, 60), (4, 384), (289, 269), (251, 252), (399, 379), (207, 227), (40, 59), (111, 110), (343, 344), (381, 361), (299, 319), (52, 51), (284, 285), (240, 260), (133, 134), (37, 17), (103, 102), (364, 384), (74, 75), (106, 107), (317, 318), (177, 176), (226, 227), (388, 387), (70, 50), (258, 259), (107, 108), (269, 268), (130, 110), (318, 319), (350, 351), (77, 76), (71, 51), (210, 209), (119, 100), (80, 81), (291, 292), (21, 41), (80, 99), (383, 384), (44, 24), (110, 109), (255, 235), (81, 82), (205, 225), (54, 74), (45, 25), (155, 135), (146, 166), (265, 266), (354, 353), (114, 115), (137, 117), (96, 76), (357, 358), (336, 335), (206, 207), (229, 209), (55, 56), (179, 199), (298, 299), (28, 48), (170, 150), (358, 359), (88, 108), (129, 109), (390, 391), (328, 327), (111, 91), (29, 49), (331, 332), (61, 81), (203, 183), (391, 392), (162, 142), (52, 32), (295, 275), (364, 365), (181, 182), (

In [ ]:
lamda = 3 # Number of colors

seed(21)

solvers = ["glucose"]

lst_x_axis = []
lst_stamps = []

print("intervals:", 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1))

for solver_name in solvers:

    transitions = CFTPsolver_ising(graph, lamda, solver_name)

    coloring1 = graph.find_coloring()
    coloring2 = coloring1.copy()
    for i in range(n):
        #coloring2[i] = '?'
       coloring2[i] = 1 - coloring1[i]
    coloring1 = graph.heatbath_dynamics(coloring1, transitions)
    coloring2 = graph.heatbath_dynamics(coloring2, transitions)
    #print(transitions)
    print(coloring1 == coloring2, "should be True")
    print_grid(coloring1,m)
    print_grid(coloring2,m)

intervals: 0.012195121951219513 0.1 0.5 0.9 0.9878048780487805
1
SAT solver time: 7.343292236328125e-05
FEASIBLE
False should be False
0.0010483264923095703
2
SAT solver time: 5.14984130859375e-05
FEASIBLE
False should be False
0.0010707378387451172
4
SAT solver time: 5.507469177246094e-05
FEASIBLE
False should be False
0.0009531974792480469
8
SAT solver time: 8.797645568847656e-05
FEASIBLE
False should be False
0.010516881942749023
16
SAT solver time: 8.368492126464844e-05
FEASIBLE
False should be False
0.0067310333251953125
32
SAT solver time: 9.512901306152344e-05
FEASIBLE
False should be False
0.0015685558319091797
64
SAT solver time: 0.00013446807861328125
FEASIBLE
False should be False
0.0037217140197753906
128
SAT solver time: 0.00023818016052246094
FEASIBLE
False should be False
0.004125118255615234
256
SAT solver time: 0.0005426406860351562
FEASIBLE
False should be False
0.017943620681762695
512
SAT solver time: 0.0009343624114990234
FEASIBLE
False should be False
0.0162105560

In [ ]:
seed(21)

lamda=3
print("intervals:", 1/(lamda**4+1), 1/(lamda**2+1), 1/2, 1-1/(lamda**2+1), 1-1/(lamda**4+1))

ret_value = 1
transitions = []
t = 1
while(ret_value is not None):
    print(t)
    while(len(transitions)<t):
        # Select a random "Glauber" transition
        node = randrange(n)
        u = random.uniform(0, 1)
        interval = find_interval(u,lamda)
        transitions.append((node, interval))

    time_before = time.time()
    coloring1 = [0 for _ in range(n)]
    coloring2 = [1 for _ in range(n)]
    coloring1out = graph.heatbath_dynamics(coloring1, transitions)
    coloring2out = graph.heatbath_dynamics(coloring2, transitions)
    print("time:", time.time() - time_before)

    if(coloring1out == coloring2out):
        ret_value = None
    t*=2

print(t)
print_grid(coloring1out, m)

intervals: 0.012195121951219513 0.1 0.5 0.9 0.9878048780487805
1
time: 7.796287536621094e-05
2
time: 9.274482727050781e-05
4
time: 8.225440979003906e-05
8
time: 8.320808410644531e-05
16
time: 0.00014400482177734375
32
time: 0.00014209747314453125
64
time: 0.00019288063049316406
128
time: 0.00033974647521972656
256
time: 0.000858306884765625
512
time: 0.0016827583312988281
1024
time: 0.0025734901428222656
2048
time: 0.006536006927490234
4096
time: 0.010834455490112305
8192
time: 0.020696163177490234
16384
time: 0.03888511657714844
32768
time: 0.07439184188842773
65536
time: 0.16821575164794922
131072
time: 0.2878551483154297
262144
time: 0.5830698013305664
524288
time: 1.2744948863983154
1048576
1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 
0 1 1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 
0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 
0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 1 
1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 
1 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 
1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 
1 0 1 1 0

In [ ]:

print(formula)

def parse_expression_to_clauses(expression):
    # Define the variable map
    variable_map = {'w': 1, 'x': 2, 'y': 3, 'z': 4, 'a': 5}
    # Remove spaces for easier processing
    expression = expression.replace(" ", "")
    # Split by '&' to get individual clauses
    clauses = expression.split('&')
    result = []

    for clause in clauses:
        # Remove outer parentheses
        clause = clause.strip('()')
        # Split by '|'
        literals = clause.split('|')
        clause_list = []

        for literal in literals:
            # Handle negation
            if literal.startswith('~'):
                clause_list.append(-variable_map[literal[1]])  # Negated variable
            else:
                clause_list.append(variable_map[literal])  # Positive variable
        result.append(clause_list)

    return result



# Input expression
expression = "(a|w|x|y)&(a|w|x|z)&(a|w|y|z)&(a|x|y|z)&(~a|~w|~x)&(~a|~w|~y)&(~a|~w|~z)&(~a|~x|~y)&(~a|~x|~z)&(~a|~y|~z)"

# Parse the expression
clauses = parse_expression_to_clauses(expression)
print(clauses)

(a | w | x | y) & (a | w | x | z) & (a | w | y | z) & (a | x | y | z) & (~a | ~w | ~x) & (~a | ~w | ~y) & (~a | ~w | ~z) & (~a | ~x | ~y) & (~a | ~x | ~z) & (~a | ~y | ~z)
[[5, 1, 2, 3], [5, 1, 2, 4], [5, 1, 3, 4], [5, 2, 3, 4], [-5, -1, -2], [-5, -1, -3], [-5, -1, -4], [-5, -2, -3], [-5, -2, -4], [-5, -3, -4]]


In [ ]:
if x==0:
            x1 = "'"
        else: x1=''
        if y==0:
            y1 = "'"
        else: y1=''
        if z==0:
            z1 = "'"
        else: z1=''
        if w==0:
            w1 = "'"
        else: w1=''
        if A==0:
            A1 = "'"
        else: A1=''
        print("x",x1 , "y", y1,"z", z1,"w",w1, "A", A1,"+", sep ='' , end='')
variables = [x,y,z,w,A]
print()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)